In [10]:
import pandas as pd
import numpy as np

# Load raw CSV
raw_path = "../data/amazon_products_sales.csv"
df = pd.read_csv(raw_path)

print("Raw shape:", df.shape)
df.head()


Raw shape: (42675, 16)


,title,rating,number_of_reviews,bought_in_last_month,current/discounted_price,price_on_variant,listed_price,is_best_seller,is_sponsored,is_couponed,buy_box_availability,delivery_details,sustainability_badges,image_url,product_url,collected_at
0,BOYA BOYALINK 2 Wireless Lavalier Microphone f...,4.6 out of 5 stars,375,300+ bought in past month,89.68,basic variant price: 2.4GHz,$159.00,No Badge,Sponsored,Save 15% with coupon,Add to cart,"Delivery Mon, Sep 1",Carbon impact,https://m.media-amazon.com/images/I/71pAqiVEs3...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
1,"LISEN USB C to Lightning Cable, 240W 4 in 1 Ch...",4.3 out of 5 stars,"2,457",6K+ bought in past month,9.99,basic variant price: nan,$15.99,No Badge,Sponsored,No Coupon,Add to cart,"Delivery Fri, Aug 29",NaN,https://m.media-amazon.com/images/I/61nbF6aVIP...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
2,"DJI Mic 2 (2 TX + 1 RX + Charging Case), Wirel...",4.6 out of 5 stars,"3,044",2K+ bought in past month,314.00,basic variant price: nan,$349.00,No Badge,Sponsored,No Coupon,Add to cart,"Delivery Mon, Sep 1",NaN,https://m.media-amazon.com/images/I/61h78MEXoj...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
3,"Apple AirPods Pro 2 Wireless Earbuds, Active N...",4.6 out of 5 stars,"35,882",10K+ bought in past month,NaN,basic variant price: $162.24,No Discount,Best Seller,Organic,No Coupon,NaN,NaN,NaN,https://m.media-amazon.com/images/I/61SUj2aKoE...,/Apple-Cancellation-Transparency-Personalized-...,2025-08-21 11:14:29
4,Apple AirTag 4 Pack. Keep Track of and find Yo...,4.8 out of 5 stars,"28,988",10K+ bought in past month,NaN,basic variant price: $72.74,No Discount,No Badge,Organic,No Coupon,NaN,NaN,NaN,https://m.media-amazon.com/images/I/61bMNCeAUA...,/Apple-MX542LL-A-AirTag-Pack/dp/B0D54JZTHY/ref...,2025-08-21 11:14:29


1. Clean Numeric Columns
- Convert rating → float ("4.6 out of 5 stars" → 4.6).
- Convert number_of_reviews → int ("35,882" → 35882).
- Convert bought_in_last_month → int ("10K+ bought" → 10000).
- Convert price columns to floats (remove $, handle NaN).



2. Clean Categorical Columns
- Convert is_best_seller, is_sponsored, is_couponed → binary (Yes/No → 1/0).
- Handle missing values (imputation strategy).


3. Feature Engineering
- Extract brand keywords from title.
- Create discount feature = listed_price – discounted_price.
- Parse collected_at → datetime features (month, day, etc).

In [11]:

# Clean Numeric Columns
# Rating: "4.6 out of 5 stars" -> 4.6
df["rating"] = (
    df["rating"]
    .astype(str)
    .str.extract(r"(\d+\.\d+)") # extract first float
    .astype(float)
)

# number_of_reviews: "35,882" -> 35882
df["number_of_reviews"] = (
    df["number_of_reviews"]
    .astype(str)
    .str.replace(",", "", regex=True) # drop commas
    .str.extract(r"(\d+)") # keep only digits
    .astype(float)
    .astype("Int64") # nullable integer
)

# bought_in_last_month: "10K+ bought" -> 10000
def parse_bought(x):
    if pd.isna(x):
        return np.nan
    x = str(x).strip()
    if "K" in x:  
        try:
            return int(float(x.replace("K+", "").replace("K", "")) * 1000)
        except:
            return np.nan
    digits = "".join(filter(str.isdigit, x))
    return int(digits) if digits else np.nan

df["bought_in_last_month"] = df["bought_in_last_month"].apply(parse_bought)

# Price columns: remove $ and keep only numeric part
for col in ["current/discounted_price", "listed_price"]:
    if col in df.columns:
        df[col] = (
            df[col]
            .astype(str)
            .str.replace(r"[^0-9\.]", "", regex=True) # allow digits & dot
            .replace("", np.nan) # blank -> NaN
            .astype(float)
        )

print("Cleaned numeric columns")
df.head()


Cleaned numeric columns


,title,rating,number_of_reviews,bought_in_last_month,current/discounted_price,price_on_variant,listed_price,is_best_seller,is_sponsored,is_couponed,buy_box_availability,delivery_details,sustainability_badges,image_url,product_url,collected_at
0,BOYA BOYALINK 2 Wireless Lavalier Microphone f...,4.6,375,300.0,89.68,basic variant price: 2.4GHz,159.00,No Badge,Sponsored,Save 15% with coupon,Add to cart,"Delivery Mon, Sep 1",Carbon impact,https://m.media-amazon.com/images/I/71pAqiVEs3...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
1,"LISEN USB C to Lightning Cable, 240W 4 in 1 Ch...",4.3,2457,NaN,9.99,basic variant price: nan,15.99,No Badge,Sponsored,No Coupon,Add to cart,"Delivery Fri, Aug 29",NaN,https://m.media-amazon.com/images/I/61nbF6aVIP...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
2,"DJI Mic 2 (2 TX + 1 RX + Charging Case), Wirel...",4.6,3044,NaN,314.00,basic variant price: nan,349.00,No Badge,Sponsored,No Coupon,Add to cart,"Delivery Mon, Sep 1",NaN,https://m.media-amazon.com/images/I/61h78MEXoj...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29
3,"Apple AirPods Pro 2 Wireless Earbuds, Active N...",4.6,35882,NaN,NaN,basic variant price: $162.24,NaN,Best Seller,Organic,No Coupon,NaN,NaN,NaN,https://m.media-amazon.com/images/I/61SUj2aKoE...,/Apple-Cancellation-Transparency-Personalized-...,2025-08-21 11:14:29
4,Apple AirTag 4 Pack. Keep Track of and find Yo...,4.8,28988,NaN,NaN,basic variant price: $72.74,NaN,No Badge,Organic,No Coupon,NaN,NaN,NaN,https://m.media-amazon.com/images/I/61bMNCeAUA...,/Apple-MX542LL-A-AirTag-Pack/dp/B0D54JZTHY/ref...,2025-08-21 11:14:29


In [12]:

#Clean Categorical Columns

# Convert Yes/No -> 1/0
binary_cols = ["is_best_seller", "is_sponsored", "is_couponed"]

for col in binary_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().str.lower()
        df[col] = df[col].map({"yes": 1, "no": 0, "1": 1, "0": 0})
        df[col] = df[col].fillna(0).astype(int)

# Handle missing values
df = df.fillna({
    "rating": df["rating"].median(),
    "number_of_reviews": df["number_of_reviews"].median(),
    "bought_in_last_month": 0,
    "current/discounted_price": df["current/discounted_price"].median(),
    "listed_price": df["listed_price"].median(),
})


In [13]:

# Feature Engineering

# Discount = listed_price – discounted_price
if "current/discounted_price" in df.columns and "listed_price" in df.columns:
    df["discount"] = df["listed_price"] - df["current/discounted_price"]

# Extract brand (first word from title as a proxy)
df["brand"] = df["title"].astype(str).str.split().str[0]

# Parse collected_at to datetime
if "collected_at" in df.columns:
    df["collected_at"] = pd.to_datetime(df["collected_at"], errors="coerce")
    df["collected_month"] = df["collected_at"].dt.month
    df["collected_day"] = df["collected_at"].dt.day
    df["collected_year"] = df["collected_at"].dt.year


In [14]:

# Save Processed Data
output_path = "../data/processed_amazon_products.csv"
df.to_csv(output_path, index=False)

print(f"Processed dataset saved to {output_path}")
print("Final shape:", df.shape)
df.head()


Processed dataset saved to ../data/processed_amazon_products.csv
Final shape: (42675, 21)


,title,rating,number_of_reviews,bought_in_last_month,current/discounted_price,price_on_variant,listed_price,is_best_seller,is_sponsored,is_couponed,...,delivery_details,sustainability_badges,image_url,product_url,collected_at,discount,brand,collected_month,collected_day,collected_year
0,BOYA BOYALINK 2 Wireless Lavalier Microphone f...,4.6,375,300.0,89.68,basic variant price: 2.4GHz,159.00,0,0,0,...,"Delivery Mon, Sep 1",Carbon impact,https://m.media-amazon.com/images/I/71pAqiVEs3...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29,69.32,BOYA,8,21,2025
1,"LISEN USB C to Lightning Cable, 240W 4 in 1 Ch...",4.3,2457,0.0,9.99,basic variant price: nan,15.99,0,0,0,...,"Delivery Fri, Aug 29",NaN,https://m.media-amazon.com/images/I/61nbF6aVIP...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29,6.00,LISEN,8,21,2025
2,"DJI Mic 2 (2 TX + 1 RX + Charging Case), Wirel...",4.6,3044,0.0,314.00,basic variant price: nan,349.00,0,0,0,...,"Delivery Mon, Sep 1",NaN,https://m.media-amazon.com/images/I/61h78MEXoj...,/sspa/click?ie=UTF8&spc=MTo4NzEzNDY2NTQ5NDYxND...,2025-08-21 11:14:29,35.00,DJI,8,21,2025
3,"Apple AirPods Pro 2 Wireless Earbuds, Active N...",4.6,35882,0.0,77.00,basic variant price: $162.24,59.99,0,0,0,...,NaN,NaN,https://m.media-amazon.com/images/I/61SUj2aKoE...,/Apple-Cancellation-Transparency-Personalized-...,2025-08-21 11:14:29,-17.01,Apple,8,21,2025
4,Apple AirTag 4 Pack. Keep Track of and find Yo...,4.8,28988,0.0,77.00,basic variant price: $72.74,59.99,0,0,0,...,NaN,NaN,https://m.media-amazon.com/images/I/61bMNCeAUA...,/Apple-MX542LL-A-AirTag-Pack/dp/B0D54JZTHY/ref...,2025-08-21 11:14:29,-17.01,Apple,8,21,2025
